In [1]:
import numpy as np
import pandas as pd
DATA_PATH = '../dataset/electricity_cost_dataset.csv'

df = pd.read_csv(DATA_PATH)

In [2]:
# Now I will proceed with each structure type seperately
df = df.drop(['recycling rate', 'air qality index', 'issue reolution time'], axis=1)

res_df = df[df['structure type'] == 'Residential']
com_df = df[df['structure type'] == 'Commercial']
ind_df = df[df['structure type'] == 'Industrial']
mix_df = df[df['structure type'] == 'Mixed-use']

res_df = res_df.drop(['structure type'], axis=1)
com_df = com_df.drop(['structure type', 'resident count'], axis=1)
ind_df = ind_df.drop(['structure type', 'resident count'], axis=1)
mix_df = mix_df.drop(['structure type'], axis=1)

train_size = 0.8
test_size = 0.2

In [3]:
# Splitting the Residential Dataset

# Shuffling the res_df dataset
shuffled_res_df = res_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Calculating the split index and splitting the data accordingly
sp_index = int(len(res_df) * train_size)
res_df_train = shuffled_res_df[:sp_index]
res_df_test = shuffled_res_df[sp_index:]

res_X_train = res_df_train.drop('electricity cost', axis=1)
res_Y_train = res_df_train['electricity cost']
res_X_test = res_df_test.drop('electricity cost', axis=1)
res_Y_test = res_df_test['electricity cost']